# Delta Lake tables 
Use this notebook to explore Delta Lake functionality 


In [1]:
from pyspark.sql.types import StructType, IntegerType, StringType, DoubleType

# define the schema
schema = StructType() \
.add("ProductID", IntegerType(), True) \
.add("ProductName", StringType(), True) \
.add("Category", StringType(), True) \
.add("ListPrice", DoubleType(), True)

df = spark.read.format("csv").option("header","true").schema(schema).load("Files/products/products.csv")
# df now is a Spark DataFrame containing CSV data from "Files/products/products.csv".
display(df)

StatementMeta(, 66f00f50-4f5e-4aad-8ded-e24e02b2c3c8, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8d819754-9ced-4436-9cc5-a49166510efa)

# Create Delta tables

### Create a managed table

In [2]:
df.write.format("delta").saveAsTable("managed_products")

StatementMeta(, 66f00f50-4f5e-4aad-8ded-e24e02b2c3c8, 4, Finished, Available, Finished)

### Create an external table

In [3]:
df.write.format("delta").saveAsTable("external_products", path="abfss://DFmm01@onelake.dfs.fabric.microsoft.com/dflh01.Lakehouse/Files/external_products")

StatementMeta(, 66f00f50-4f5e-4aad-8ded-e24e02b2c3c8, 5, Finished, Available, Finished)

### Compare managed and external tables

In [4]:
%%sql
DESCRIBE FORMATTED managed_products;

StatementMeta(, 66f00f50-4f5e-4aad-8ded-e24e02b2c3c8, 6, Finished, Available, Finished)

<Spark SQL result set with 12 rows and 3 fields>

In [5]:
%%sql
DESCRIBE FORMATTED external_products;

StatementMeta(, 66f00f50-4f5e-4aad-8ded-e24e02b2c3c8, 7, Finished, Available, Finished)

<Spark SQL result set with 12 rows and 3 fields>

In [6]:
%%sql
DROP TABLE managed_products;
DROP TABLE external_products;

StatementMeta(, 66f00f50-4f5e-4aad-8ded-e24e02b2c3c8, 9, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

## Use SQL to create a Delta table

In [8]:
%%sql
CREATE TABLE products
USING DELTA
LOCATION 'Files/external_products';

StatementMeta(, 66f00f50-4f5e-4aad-8ded-e24e02b2c3c8, 11, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [9]:
%%sql
SELECT * FROM products;

StatementMeta(, 66f00f50-4f5e-4aad-8ded-e24e02b2c3c8, 12, Finished, Available, Finished)

<Spark SQL result set with 295 rows and 4 fields>

##  Explore table versioning

In [1]:
%%sql
UPDATE products
SET ListPrice = ListPrice * 0.9
WHERE Category = 'Mountain Bikes';

StatementMeta(, 159253c2-5897-4835-8fd8-510a91373759, 2, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 1 fields>

In [2]:
%%sql
DESCRIBE HISTORY products;

StatementMeta(, 159253c2-5897-4835-8fd8-510a91373759, 3, Finished, Available, Finished)

<Spark SQL result set with 2 rows and 15 fields>

In [3]:
delta_table_path = 'Files/external_products'
# Get the current data
current_data = spark.read.format("delta").load(delta_table_path)
display(current_data)

# Get the version 0 data
original_data = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
display(original_data)

StatementMeta(, 159253c2-5897-4835-8fd8-510a91373759, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4da4f47f-8155-44b4-a8db-2f56230a21bf)

SynapseWidget(Synapse.DataFrame, dee78cd7-a1db-4ac2-9973-a07862fe005d)

## Analyze Delta table data with SQL queries

In [4]:
%%sql
-- Create a temporary view
CREATE OR REPLACE TEMPORARY VIEW products_view
AS
   SELECT Category, COUNT(*) AS NumProducts, MIN(ListPrice) AS MinPrice, MAX(ListPrice) AS MaxPrice, AVG(ListPrice) AS AvgPrice
      FROM products
         GROUP BY Category;

         SELECT *
         FROM products_view
         ORDER BY Category;    

StatementMeta(, 159253c2-5897-4835-8fd8-510a91373759, 7, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 37 rows and 5 fields>

In [5]:
%%sql
SELECT Category, NumProducts
FROM products_view
ORDER BY NumProducts DESC
LIMIT 10;

StatementMeta(, 159253c2-5897-4835-8fd8-510a91373759, 8, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 2 fields>

In [6]:
from pyspark.sql.functions import col, desc

df_products = spark.sql("SELECT Category, MinPrice, MaxPrice, AvgPrice FROM products_view").orderBy(col("AvgPrice").desc())
display(df_products.limit(6))

StatementMeta(, 159253c2-5897-4835-8fd8-510a91373759, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fe8df30a-2e49-4cdb-ad94-0a93436082eb)

# Use Delta tables for streaming data

In [7]:
from notebookutils import mssparkutils
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Create a folder
inputPath = 'Files/data/'
mssparkutils.fs.mkdirs(inputPath)

# Create a stream that reads data from the folder, using a JSON schema
jsonSchema = StructType([
StructField("device", StringType(), False),
StructField("status", StringType(), False)
])
iotstream = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(inputPath)

# Write some event data to the folder
device_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''

mssparkutils.fs.put(inputPath + "data.txt", device_data, True)

print("Source stream created...")

StatementMeta(, 159253c2-5897-4835-8fd8-510a91373759, 10, Finished, Available, Finished)

Source stream created...


In [8]:
# Write the stream to a delta table
delta_stream_table_path = 'Tables/iotdevicedata'
checkpointpath = 'Files/delta/checkpoint'
deltastream = iotstream.writeStream.format("delta").option("checkpointLocation", checkpointpath).start(delta_stream_table_path)
print("Streaming to delta sink...")

StatementMeta(, 159253c2-5897-4835-8fd8-510a91373759, 11, Finished, Available, Finished)

Streaming to delta sink...


In [9]:
%%sql
SELECT * FROM IotDeviceData;

StatementMeta(, 159253c2-5897-4835-8fd8-510a91373759, 12, Finished, Available, Finished)

<Spark SQL result set with 9 rows and 2 fields>

In [10]:
# Add more data to the source stream
more_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''

mssparkutils.fs.put(inputPath + "more-data.txt", more_data, True)

StatementMeta(, 159253c2-5897-4835-8fd8-510a91373759, 13, Finished, Available, Finished)

True

In [11]:
%%sql
SELECT * FROM IotDeviceData;

StatementMeta(, 159253c2-5897-4835-8fd8-510a91373759, 14, Finished, Available, Finished)

<Spark SQL result set with 16 rows and 2 fields>

In [12]:
deltastream.stop()

StatementMeta(, 159253c2-5897-4835-8fd8-510a91373759, 15, Finished, Available, Finished)